In [1]:
import pandas as pd
from IPython.display import Image
import folium

## Satellite Imagery of Singapore

### SENTINEL-2 Data for LCZ Identification

#### Set Up Connection with Earth Engine

In [2]:
# import Google Earth Engine API
import ee
import geemap

# import custom Earth Engine functions
from gee_functions import *

In [4]:
# create authentication token for accessing Earth Engine API
ee.Authenticate()

Enter verification code:  4/1AbUR2VMgOhYtleP-JMdVNnuucn24r26rdlz0umjcdsUFPWNSWas3gM1v6-Q



Successfully saved authorization token.


In [3]:
# Initialise access to Earth Engine
ee.Initialize()

#### Get Satellite Images

As Earth Engine has a limit to the number of pixels that can be downloaded at any one time, we will use mosaic method to download the chunks and stitch it up together again

In [4]:
# Define geographical bounds of dataset
singapore_bounds = ee.FeatureCollection('USDOS/LSIB/2017').filter(ee.Filter.eq('COUNTRY_NA', 'Singapore'))
#bounds = Singapore.geometry().bounds()

In [5]:
start_date = '2022-11-09'
end_date = '2023-05-10'

In [11]:
def get_s2_w_cloud_prob(aoi):
    '''
    Function that merges 2 collections of SENTINEL 2 RGB bands
    '''
    # get the actual image in all bands
    sen2 = (ee.ImageCollection('COPERNICUS/S2_SR')
            .filterDate(START_DATE, END_DATE)
            .filterBounds(aoi)
            .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))
            #.select(['B4', 'B3', 'B2'])
           )

    # get the cloud probability scale
    sen2_cloud = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
                 .filterDate(START_DATE, END_DATE)
                 .filterBounds(aoi)
                 )
    
    # define merging parameters
    merge_params = {'primary': sen2,
                        'secondary': sen2_cloud,
                        'condition': ee.Filter.equals(**{'leftField': 'system:index', 'rightField': 'system:index'})
                       }
    
    # merge the 2 images together
    sen2_cloudless = ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**merge_params))
                 
    return sen2_cloudless


In [12]:
def add_cloud_bands(img):
    '''
    Function that adds the cloud probability layer
    '''
    # subset the probability
    cloud_prob = ee.Image(img.get('s2cloudless')).select('probability')
    
    # filter only those that pass the threshold
    is_cloud = cloud_prob.gt(CLOUD_PROB_THRES).rename('clouds')
    
    # add a new variable
    return img.addBands(ee.Image([cloud_prob, is_cloud]))
    

In [13]:
def add_shadow_bands(img):
    '''
    Function that adds the shadow probability layer
    '''
    # identify non-water pixels
    not_water = img.select('SCL').neq(6)
    
    # identify dark near-infra that are not water
    sr_band_scale = 1e4
    dark_pixels = img.select('B8').lt(NIR_DARK_THRES*sr_band_scale).multiply(not_water).rename('dark_pixels')
    
    # determine direction to project cloud shadow from clouds
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')))
    
    # project shadows from clouds for distance from clouds
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLOUD_PROJ_DIST*10)
                .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
                .select('distance')
                .mask()
                .rename('cloud_transform')
               )
    
    # identify intersection of dark pixels with cloud shadow projection
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')
    
    # add to main img
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

In [14]:
def add_cloud_shadow_mask(img):
    '''
    Function that assemble the cloud and shadow components into a single mask
    '''
    # add cloud band
    img_cloud = add_cloud_bands(img)
    
    # add shadows
    img_cloud_shadows = add_shadow_bands(img_cloud)
    
    # combine, set cloud and shadow as value 1, else 0
    is_cld_shdw = img_cloud_shadows.select('clouds').add(img_cloud_shadows.select('shadows')).gt(0)
    
    # remove small cloud patches and dilate remaining pixels by buff input
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/10) # 10m scale
                   .reproject(**{'crs': img.select([0]).projection(), 'scale':10})
                   .rename('cloudmask')
                  ) 
    
    # add final cloud mask
    return img_cloud_shadows.addBands(is_cld_shdw)
    

In [15]:
def apply_cld_shdw_mask(img):
    '''
    Function that apply cloud mask to each image in collection
    '''
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)


In [6]:
# tie everything together
sen2_cloudless = get_s2_w_cloud_prob(singapore_bounds, start_date, end_date)

sen2_cloudless_median = (sen2_cloudless.map(add_cloud_shadow_mask)
                                       .map(apply_cld_shdw_mask)
                                       .median())

In [11]:
sen2_list = sen2.getRegion(Singapore.geometry().bounds(), 1000)

In [10]:
def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['longitude', 'latitude' ,'datetime',  *list_of_bands]]

    return df

In [183]:
ee_array_to_df(sen2_list.getInfo(), ['B4', 'B3', 'B2'])

,longitude,latitude,datetime,B4,B3,B2
0,103.605846,1.214971,2023-05-05 03:37:49.805,1709,2049,2299
1,103.606744,1.214971,2023-05-05 03:37:49.805,1696,2005,2251
2,103.607642,1.214971,2023-05-05 03:37:49.805,1722,2041,2293
3,103.608541,1.214971,2023-05-05 03:37:49.805,1725,2083,2327
4,103.609439,1.214971,2023-05-05 03:37:49.805,1719,2098,2339
...,...,...,...,...,...,...
152470,104.081953,1.470093,2023-05-05 03:37:49.805,3595,4053,4344
152471,104.082851,1.470093,2023-05-05 03:37:49.805,2329,2767,2930
152472,104.08375,1.470093,2023-05-05 03:37:49.805,3817,4173,4388
152473,104.084648,1.470093,2023-05-05 03:37:49.805,5672,6182,6739


In [14]:
sen2_list.getInfo()

[['id', 'longitude', 'latitude', 'time', 'B4', 'B3', 'B2'],
 ['20230505T031519_20230505T033023_T48NUG',
  103.60719329392502,
  1.2172172099819516,
  1683257869805,
  1693,
  2037,
  2270],
 ['20230505T031519_20230505T033023_T48NUG',
  103.61617644676622,
  1.2172172099819516,
  1683257869805,
  2051,
  2345,
  2493],
 ['20230505T031519_20230505T033023_T48NUG',
  103.6251595996074,
  1.2172172099819516,
  1683257869805,
  3842,
  3847,
  3724],
 ['20230505T031519_20230505T033023_T48NUG',
  103.6341427524486,
  1.2172172099819516,
  1683257869805,
  4138,
  3875,
  3474],
 ['20230505T031519_20230505T033023_T48NUG',
  103.64312590528979,
  1.2172172099819516,
  1683257869805,
  2566,
  2700,
  2560],
 ['20230505T031519_20230505T033023_T48NUG',
  103.65210905813099,
  1.2172172099819516,
  1683257869805,
  1611,
  1826,
  1927],
 ['20230505T031519_20230505T033023_T48NUG',
  103.66109221097219,
  1.2172172099819516,
  1683257869805,
  1647,
  1966,
  2040],
 ['20230505T031519_20230505T0330

In [19]:
sen2_cloudless_median.select(['B4', 'B3', 'B2'])

In [21]:
sen2_cloudless

Name,Description
B1,Aerosols
B2,Blue
B3,Green
B4,Red
B5,Red Edge 1
B6,Red Edge 2
B7,Red Edge 3
B8,NIR
B8A,Red Edge 4
B9,Water vapor


In [17]:
# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [22]:
# Create a folium map object.
center = singapore_bounds.geometry().bounds().centroid(10).coordinates().reverse().getInfo()
m = folium.Map(location=center, zoom_start=12)

# Add layers to the folium map.
m.add_ee_layer(sen2_cloudless_median,
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 4000, 'gamma': 1.1},
                'S2 cloud-free mosaic', True, 1, 9)

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)


In [67]:
url = sen2_cloudless_median.select(['B4', 'B3', 'B2']).getThumbUrl({'max':12000, #'dimensions':512,
                                                                    'scale':100, 'format':'jpg'})
Image(url=url, width=1024)

EEException: Pixel grid dimensions (400752x200376) must be less than or equal to 32768.

In [60]:
url

'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/dc4ebd67f635f38531c69397b4478b2a-af82433304dd90925502268bdd2bc45c:getPixels'

In [29]:
url = sen2_cloudless.median().getThumbUrl({'max':12000, #'dimensions':512,
                               'region':Singapore.geometry().bounds(), 'scale':100, 'format':'jpg'})
Image(url=url, width=1024)

In [240]:
# Define the region of interest
#Singapore = ee.Geometry.Point(103.8, 1.3).buffer(10000)

# Load the Sentinel-2 image collection
#sen2 = ee.ImageCollection('COPERNICUS/S2_SR').filterDate('2023-04-28','2023-05-09').select(['B4', 'B3', 'B2'])

# Define the scale and tile dimensions
scale = 100
tileWidth = 1000
tileHeight = 1000

# Get the bounds of the region of interest
bounds = Singapore.geometry().bounds()

# Compute the number of tiles needed in each direction
numCols = ee.Number(bounds.getInfo()['coordinates'][0][2][0]).subtract(ee.Number(bounds.getInfo()['coordinates'][0][0][0])).divide(scale*tileWidth).ceil()
numRows = ee.Number(bounds.getInfo()['coordinates'][0][2][1]).subtract(ee.Number(bounds.getInfo()['coordinates'][0][0][1])).divide(scale*tileHeight).ceil()

# Create a list of tiles
tiles = []
for col in range(numCols.getInfo()):
    for row in range(numRows.getInfo()):
        tileBounds = ee.Geometry.Rectangle(
            [ee.Number(bounds.coordinates().getInfo()[0][0][0]).add(col*scale*tileWidth), 
             ee.Number(bounds.coordinates().getInfo()[0][0][1]).add(row*scale*tileHeight), 
             ee.Number(bounds.coordinates().getInfo()[0][0][0]).add((col+1)*scale*tileWidth), 
             ee.Number(bounds.coordinates().getInfo()[0][0][1]).add((row+1)*scale*tileHeight)])
        tile = sen2.filterBounds(tileBounds).mosaic().clip(tileBounds)
        tiles.append(tile)

# Mosaic the tiles back together
mosaic = ee.ImageCollection(tiles).mosaic()

# Display the result
print(mosaic.getInfo())

{'type': 'Image', 'bands': [{'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [11]:

# Define the scale and tile dimensions
scale = 10 # 10m resolution
tileWidth = 32 # 32 pixels per tile
tileHeight = 32 # 32 pixels per tile, i.e. square tile

# compute number of tiles needed in each direction
numCols = ee.Number(bounds.getInfo()['coordinates'][0][2][0]).subtract(ee.Number(bounds.getInfo()['coordinates'][0][0][0])).divide(scale*tileWidth).ceil()
numRows = ee.Number(bounds.getInfo()['coordinates'][0][2][1]).subtract(ee.Number(bounds.getInfo()['coordinates'][0][0][1])).divide(scale*tileHeight).ceil()

In [24]:
ee.Image.retile()

AttributeError: module 'ee.geometry' has no attribute 'tile'

In [19]:
(bounds.getInfo()['coordinates'][0][2][0] - bounds.getInfo()['coordinates'][0][0][0])/(scale*tileWidth)

0.001500557167893124

In [18]:
ee.Number(bounds.getInfo()['coordinates'][0][2][0]).subtract(ee.Number(bounds.getInfo()['coordinates'][0][0][0])).getInfo()

0.4801782937257997

In [241]:
mosaic.reproject(crs='EPSG:3414', scale=scale).getThumbUrl(params={'dimensions':500, 'format':'png'})

'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/836439fe09a177dfb3174dbfbf1beb55-a76a9b10c8c6bf585813eb0d6129934c:getPixels'

In [230]:
ee.Number(bounds.coordinates().getInfo()[0][0][0]).add(1).getInfo()

104.60552745357612